In [1]:
from mmdet.apis import init_detector
import mmcv
from mmcv import Config


import copy
import os.path as osp

import numpy as np

from mmdet.datasets.builder import DATASETS
from mmdet.datasets.custom import CustomDataset

from mmdet.apis import set_random_seed


import json

/opt/conda/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import glob as _glob
import os
    
def glob(dir, pats, recursive=False):  # faster than match, python3 only
    pats = pats if isinstance(pats, (list, tuple)) else [pats]
    matches = []
    for pat in pats:
        matches += _glob.glob(os.path.join(dir, pat), recursive=recursive)
    return matches


In [3]:
#### load_annotations에서 뒤의 변수 받는거 custom dataset 에서는 이름을 바꿔도 되지만 아래에
#### configuration에서는 무조건 변수명을 ann_file로 받아야함
@DATASETS.register_module()
class Drive_dataset(CustomDataset):
    CLASSES=('car','bus','truck', 'special vehicle', 'motorcycle','bicycle','personal mobility','person','Traffic_light', 'Traffic_sign')


    def load_annotations(self, ann_fol):
        
        CLASSES_dict = {'car' : 0 , 'bus' : 1, 'truck' : 2, 'special vehicle' : 3, 'motorcycle' : 4,'bicycle' : 5 ,'personal mobility' : 6 
                        ,'person' : 7 ,'Traffic_light' : 8, 'Traffic_sign' : 9}
        
        cat2label = {k: i for i, k in enumerate(self.CLASSES)}
        
        data_infos = []
        
        ls = glob(ann_fol,'*',True)
        
        for idx,an in enumerate(ls):
            json_data = {}
            with open(an, "r") as json_file:
                json_data = json.load(json_file)
                
            ansplit = an.split('/')
            
            filename = ansplit[0] + '/' + ansplit[1] + '/' + 'images'+'/'+ json_data['image_name']
            
            width, height = json_data['image_size']

            data_info = dict(filename=filename, width=width, height=height)

            gt_bboxes = []
            gt_labels = []

            for ann_data in json_data['Annotation']:
                gt_labels.append(CLASSES_dict[ann_data['class_name']])
                gt_bboxes.append(ann_data['data'])


            data_anno = dict(
                    bboxes=np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
                    labels=np.array(gt_labels, dtype=np.long))


            data_info.update(ann=data_anno)
            data_infos.append(data_info)
            
            if idx!=0 and idx%20000==0:
                print(str(idx)+'/'+str(len(ls))+' load annotations END!')
            
        
        
        return data_infos

In [4]:
## 추가수정 기존 받았던 pretrain과 매칭되는 config로 수정 
cfg = Config.fromfile('UniverseNet/configs/waymo_open/universenet50_2008_fp16_4x4_mstrain_640_1280_1x_waymo_open_f0.py') 

In [5]:
print(f'Config:\n{cfg.pretty_text}')

Config:
pretrained_ckpt = 'https://github.com/shinya7y/weights/releases/download/v1.0.2/res2net50_v1b_26w_4s-3cf99910_mmdetv2-92ed3313.pth'
model = dict(
    type='GFL',
    backbone=dict(
        type='Res2Net',
        depth=50,
        scales=4,
        base_width=26,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='SyncBN', requires_grad=True),
        norm_eval=False,
        style='pytorch',
        dcn=dict(type='DCN', deform_groups=1, fallback_on_stride=False),
        stage_with_dcn=(False, False, False, True),
        init_cfg=dict(
            type='Pretrained',
            checkpoint=
            'https://github.com/shinya7y/weights/releases/download/v1.0.2/res2net50_v1b_26w_4s-3cf99910_mmdetv2-92ed3313.pth'
        )),
    neck=[
        dict(
            type='FPN',
            in_channels=[256, 512, 1024, 2048],
            out_channels=256,
            start_level=1,
            add_extra_convs='on_output',
   

In [6]:
## 추가 및 수정 ## 
cfg.dataset_type  = 'Drive_dataset'
cfg.data_root = ''

## single GPU 이기 때문에 syncBN 이 아닌 BN으로 수정)
cfg.model.backbone.norm_cfg=dict(type='BN', requires_grad=True)

## 기존 UniverseNet에서 부른거 올려두고 4070 suepr가 감당할 수 있는 수준으로 수정하기
cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True),
    dict(
        type='Resize',
        img_scale=[(960, 640), (1920, 1200)],
        multiscale_mode='range',
        keep_ratio=True),
    dict(
        type='PhotoMetricDistortion',
        brightness_delta=32,
        contrast_range=(0.5, 1.5),
        saturation_range=(0.5, 1.5),
        hue_delta=18),
     dict(
        type='RandomCrop',
        crop_type='absolute_range',
        crop_size=(640, 960)),  ##### crop size는 (height, width) 로 되있음
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img', 'gt_bboxes', 'gt_labels'])
    
]

cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(
                type='MultiScaleFlipAug',
                img_scale=(1920, 1200),
                flip=False,
                transforms=[
                    dict(type='Resize', keep_ratio=True),
                    dict(type='RandomFlip'),
                    dict(
                        type='Normalize',
                        mean=[123.675, 116.28, 103.53],
                        std=[58.395, 57.12, 57.375],
                        to_rgb=True),
                    dict(type='Pad', size_divisor=32),
                    dict(type='ImageToTensor', keys=['img']),
                    dict(type='Collect', keys=['img'])
                ])
]

cfg.data=dict(
    samples_per_gpu=10,
    workers_per_gpu=12,
    train=dict(
        type=cfg.dataset_type,
        ann_file='2DBB/training/labels/',
        pipeline=cfg.train_pipeline),
     val=dict(
        type=cfg.dataset_type,
        ann_file='2DBB/validation/labels',
        pipeline=cfg.test_pipeline),
    test=None)

cfg.model.bbox_head.num_classes=10

cfg.device='cuda'
cfg.work_dir = 'checkpoints_ver1'

cfg.log_config.interval = 4000 #iteration 단위

cfg.seed = 2024

set_random_seed(cfg.seed, deterministic=False)

cfg.workflow = [('train', 1), ('val', 1)]

cfg.evaluation = dict(interval=1, metric='mAP')

cfg.load_from = 'universenet50_2008_fp16_4x4_mstrain_480_960_2x_coco_20200815_epoch_24-81356447.pth'
cfg.runner = dict(type='EpochBasedRunner', max_epochs=24)

cfg.gpu_ids = range(1)

In [7]:
print(f'Config:\n{cfg.pretty_text}')

Config:
pretrained_ckpt = 'https://github.com/shinya7y/weights/releases/download/v1.0.2/res2net50_v1b_26w_4s-3cf99910_mmdetv2-92ed3313.pth'
model = dict(
    type='GFL',
    backbone=dict(
        type='Res2Net',
        depth=50,
        scales=4,
        base_width=26,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=False,
        style='pytorch',
        dcn=dict(type='DCN', deform_groups=1, fallback_on_stride=False),
        stage_with_dcn=(False, False, False, True),
        init_cfg=dict(
            type='Pretrained',
            checkpoint=
            'https://github.com/shinya7y/weights/releases/download/v1.0.2/res2net50_v1b_26w_4s-3cf99910_mmdetv2-92ed3313.pth'
        )),
    neck=[
        dict(
            type='FPN',
            in_channels=[256, 512, 1024, 2048],
            out_channels=256,
            start_level=1,
            add_extra_convs='on_output',
       

In [8]:
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
import torch

In [ ]:

# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(cfg.model)

# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

meta=dict()
meta['config'] = cfg.pretty_text
meta['seed'] = cfg.seed
# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True,meta=meta)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:42: DeprecationWarning: `np.long` is a deprecated alias for `np.compat.long`. To silence this warning, use `np.compat.long` by itself. In the likely event your code does not need to work on Python 2 you can use the builtin `int` for which `np.compat.long` is itself an alias. Doing this will not modify any behaviour and is safe. When replacing `np.long`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations


20000/80000 load annotations END!
40000/80000 load annotations END!
60000/80000 load annotations END!


/root/UniverseNet/mmdet/datasets/custom.py:186: UserWarning: CustomDataset does not support filtering empty gt images.
  'CustomDataset does not support filtering empty gt images.')
/root/UniverseNet/mmdet/models/dense_heads/anchor_head.py:116: UserWarning: DeprecationWarning: `num_anchors` is deprecated, for consistency or also use `num_base_priors` instead
  warnings.warn('DeprecationWarning: `num_anchors` is deprecated, '
2024-08-01 05:18:05,140 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2024-08-01 05:18:21,238 - mmdet - INFO - load checkpoint from local path: universenet50_2008_fp16_4x4_mstrain_480_960_2x_coco_20200815_epoch_24-81356447.pth
2024-08-01 05:18:21,758 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for bbox_head.gfl_cls.weight: copying a param with shape torch.Size([80, 256, 3, 3]) from checkpoint, the shape in current model is torch.Size([10, 256, 3, 3]).
size mismatch for bbox_head.gfl_cls.bias:

[>>>>>>>>>>>>>>>>>>>>>>>>] 10000/10000, 15.7 task/s, elapsed: 639s, ETA:     0s
---------------iou_thr: 0.5---------------


2024-08-01 06:57:11,852 - mmdet - INFO - 
+-------------------+-------+--------+--------+-------+
| class             | gts   | dets   | recall | ap    |
+-------------------+-------+--------+--------+-------+
| car               | 32964 | 400574 | 0.979  | 0.896 |
| bus               | 979   | 30175  | 0.915  | 0.741 |
| truck             | 5874  | 194740 | 0.956  | 0.762 |
| special vehicle   | 202   | 25373  | 0.599  | 0.141 |
| motorcycle        | 235   | 4685   | 0.596  | 0.456 |
| bicycle           | 62    | 1954   | 0.435  | 0.207 |
| personal mobility | 43    | 4378   | 0.698  | 0.264 |
| person            | 7347  | 65591  | 0.865  | 0.740 |
| Traffic_light     | 3298  | 19903  | 0.915  | 0.803 |
| Traffic_sign      | 2889  | 75329  | 0.931  | 0.805 |
+-------------------+-------+--------+--------+-------+
| mAP               |       |        |        | 0.581 |
+-------------------+-------+--------+--------+-------+
2024-08-01 06:57:11,855 - mmdet - INFO - Epoch(val) [1][10000]

...

2024-08-02 04:20:26,111 - mmdet - INFO - 
+-------------------+-------+--------+--------+-------+
| class             | gts   | dets   | recall | ap    |
+-------------------+-------+--------+--------+-------+
| car               | 32964 | 268843 | 0.988  | 0.960 |
| bus               | 979   | 13790  | 0.963  | 0.916 |
| truck             | 5874  | 96841  | 0.979  | 0.917 |
| special vehicle   | 202   | 8857   | 0.891  | 0.709 |
| motorcycle        | 235   | 5359   | 0.813  | 0.715 |
| bicycle           | 62    | 1486   | 0.903  | 0.799 |
| personal mobility | 43    | 1633   | 0.907  | 0.780 |
| person            | 7347  | 65298  | 0.926  | 0.844 |
| Traffic_light     | 3298  | 23155  | 0.966  | 0.913 |
| Traffic_sign      | 2889  | 35946  | 0.967  | 0.903 |
+-------------------+-------+--------+--------+-------+
| mAP               |       |        |        | 0.846 |
+-------------------+-------+--------+--------+-------+
2024-08-02 04:20:26,116 - mmdet - INFO - Epoch(val) [14][10000

[>>>>>>>>>>>>>>>>>>>>>>>>] 10000/10000, 15.5 task/s, elapsed: 647s, ETA:     0s
---------------iou_thr: 0.5---------------


2024-08-02 05:59:11,434 - mmdet - INFO - 
+-------------------+-------+--------+--------+-------+
| class             | gts   | dets   | recall | ap    |
+-------------------+-------+--------+--------+-------+
| car               | 32964 | 258585 | 0.988  | 0.960 |
| bus               | 979   | 13432  | 0.965  | 0.917 |
| truck             | 5874  | 92238  | 0.979  | 0.917 |
| special vehicle   | 202   | 8613   | 0.886  | 0.710 |
| motorcycle        | 235   | 5543   | 0.817  | 0.719 |
| bicycle           | 62    | 1460   | 0.903  | 0.801 |
| personal mobility | 43    | 1633   | 0.884  | 0.777 |
| person            | 7347  | 63715  | 0.927  | 0.844 |
| Traffic_light     | 3298  | 22601  | 0.968  | 0.914 |
| Traffic_sign      | 2889  | 34503  | 0.967  | 0.903 |
+-------------------+-------+--------+--------+-------+
| mAP               |       |        |        | 0.846 |
+-------------------+-------+--------+--------+-------+
2024-08-02 05:59:11,438 - mmdet - INFO - Epoch(val) [15][10000

[>>>>>>>>>>>>>>>>>>>>>>>>] 10000/10000, 15.5 task/s, elapsed: 645s, ETA:     0s
---------------iou_thr: 0.5---------------


2024-08-02 07:37:54,713 - mmdet - INFO - 
+-------------------+-------+--------+--------+-------+
| class             | gts   | dets   | recall | ap    |
+-------------------+-------+--------+--------+-------+
| car               | 32964 | 270101 | 0.988  | 0.960 |
| bus               | 979   | 12770  | 0.965  | 0.918 |
| truck             | 5874  | 92030  | 0.979  | 0.918 |
| special vehicle   | 202   | 7929   | 0.891  | 0.709 |
| motorcycle        | 235   | 5360   | 0.821  | 0.719 |
| bicycle           | 62    | 1402   | 0.903  | 0.801 |
| personal mobility | 43    | 1641   | 0.907  | 0.782 |
| person            | 7347  | 65172  | 0.930  | 0.846 |
| Traffic_light     | 3298  | 23476  | 0.967  | 0.914 |
| Traffic_sign      | 2889  | 36220  | 0.969  | 0.904 |
+-------------------+-------+--------+--------+-------+
| mAP               |       |        |        | 0.847 |
+-------------------+-------+--------+--------+-------+
2024-08-02 07:37:54,717 - mmdet - INFO - Epoch(val) [16][10000